In [1]:
# Cell 1 — imports & paths
import pandas as pd
from pathlib import Path

project = Path(r"D:\SaaS Ticket Analytics Dashboard")
cleaned = project / "cleaned"
cleaned.mkdir(parents=True, exist_ok=True)

print("Cleaned folder:", cleaned)


Cleaned folder: D:\SaaS Ticket Analytics Dashboard\cleaned


In [2]:
# Cell 2 — load tickets CSV
src = cleaned / "tickets_master_clean.csv"   # if you have a raw source, change this path
print("Loading:", src)
tickets = pd.read_csv(src, low_memory=False, parse_dates=['created_at','first_response_at','resolved_at'])
tickets.shape


Loading: D:\SaaS Ticket Analytics Dashboard\cleaned\tickets_master_clean.csv


(11000, 25)

In [3]:
# Cell 3 — drop exact duplicates
print("Before duplicates:", len(tickets))
tickets = tickets.drop_duplicates()
print("After duplicates:", len(tickets))


Before duplicates: 11000
After duplicates: 11000


In [4]:
# Cell 4 — normalize column names
tickets.columns = [c.strip().lower().replace(' ', '_') for c in tickets.columns]
print("Columns normalized. Sample:", tickets.columns.tolist()[:20])


Columns normalized. Sample: ['ticket_id', 'created_at', 'first_response_at', 'resolved_at', 'priority', 'status', 'issue_type', 'channel', 'customer_id', 'agent_id', 'first_response_mins', 'resolution_time_hours', 'sla_hours', 'sla_breached', 'reopened', 'escalated', 'csat', 'created_date', 'created_hour', 'created_weekday']


In [5]:
# Cell 5 — date conversions and derived cols
tickets['created_at'] = pd.to_datetime(tickets['created_at'], errors='coerce')
tickets['first_response_at'] = pd.to_datetime(tickets['first_response_at'], errors='coerce') if 'first_response_at' in tickets.columns else None
tickets['resolved_at'] = pd.to_datetime(tickets['resolved_at'], errors='coerce') if 'resolved_at' in tickets.columns else None

tickets['created_date'] = tickets['created_at'].dt.date
tickets['year_month'] = tickets['created_at'].dt.to_period('M').astype(str)
tickets['created_hour'] = tickets['created_at'].dt.hour

print("Date columns processed. created_at min/max:", tickets['created_at'].min(), tickets['created_at'].max())


Date columns processed. created_at min/max: 2024-11-01 00:15:17 2025-10-31 22:21:02


In [6]:
# Cell 6 — compute resolution_time_hours and first_response_mins
# resolution_time_hours = (resolved_at - created_at) in hours
if 'resolved_at' in tickets.columns:
    tickets['resolution_time_hours'] = (tickets['resolved_at'] - tickets['created_at']).dt.total_seconds() / 3600.0
else:
    tickets['resolution_time_hours'] = pd.NA

# first_response_mins = (first_response_at - created_at) in minutes
if 'first_response_at' in tickets.columns:
    tickets['first_response_mins'] = (tickets['first_response_at'] - tickets['created_at']).dt.total_seconds() / 60.0
else:
    tickets['first_response_mins'] = pd.NA

# quick sanity
print("resolution_time_hours: min/max", tickets['resolution_time_hours'].min(), tickets['resolution_time_hours'].max())
print("first_response_mins: min/max", tickets['first_response_mins'].min(), tickets['first_response_mins'].max())


resolution_time_hours: min/max 1.0 145.89180547916666
first_response_mins: min/max 2.0 1174.0


In [7]:
# Cell 7 — normalize sla_breached column
# If there's an existing boolean/int/text column, convert, else create from sla_hours if available.
if 'sla_breached' in tickets.columns:
    tickets['sla_breached'] = tickets['sla_breached'].fillna(0).astype(int)
elif 'sla_hours' in tickets.columns and 'resolution_time_hours' in tickets.columns:
    tickets['sla_breached'] = (tickets['resolution_time_hours'] > tickets['sla_hours']).astype(int)
else:
    tickets['sla_breached'] = 0  # default if no SLA info

print("SLA breached value counts:\n", tickets['sla_breached'].value_counts(dropna=False).to_dict())


SLA breached value counts:
 {0: 11000}


In [8]:
# Cell 8 — clean csat values
if 'csat' in tickets.columns:
    tickets['csat'] = pd.to_numeric(tickets['csat'], errors='coerce')
    # optional: clip to plausible range 1-5
    tickets['csat'] = tickets['csat'].clip(lower=1, upper=5)
    print("CSAT stats:", tickets['csat'].describe())
else:
    print("No csat column found; skipping.")


CSAT stats: count    11000.000000
mean         3.560191
std          0.727957
min          1.000000
25%          3.100000
50%          3.600000
75%          4.100000
max          5.000000
Name: csat, dtype: float64


In [9]:
# Cell 9 — clean text categorical fields
for col in ['issue_type', 'priority', 'channel', 'agent_name', 'customer_id']:
    if col in tickets.columns:
        tickets[col] = tickets[col].astype(str).str.strip()
print("Categorical cleanup done.")


Categorical cleanup done.


In [10]:
# Cell 10 — drop rows without ticket_id or created_at
before = len(tickets)
tickets = tickets.dropna(subset=['ticket_id', 'created_at'])
after = len(tickets)
print(f"Dropped {before-after} rows with missing ticket_id or created_at.")


Dropped 0 rows with missing ticket_id or created_at.


In [11]:
# Cell 11 — recompute simple KPIs
print("Total tickets:", len(tickets))
print("Avg first response mins:", round(tickets['first_response_mins'].mean(),2))
print("Avg resolution hrs:", round(tickets['resolution_time_hours'].mean(),2))
print("SLA breach %:", round(tickets['sla_breached'].mean()*100,2) if 'sla_breached' in tickets.columns else "N/A")
print("Avg CSAT:", round(tickets['csat'].mean(),2) if 'csat' in tickets.columns else "N/A")


Total tickets: 11000
Avg first response mins: 260.39
Avg resolution hrs: 20.86
SLA breach %: 0.0
Avg CSAT: 3.56


In [12]:
# Cell 12 — save cleaned tickets CSV
out_tickets = cleaned / "tickets_master_clean.csv"
tickets.to_csv(out_tickets, index=False)
print("Saved cleaned tickets to:", out_tickets)

# If you have agents/customers DataFrames loaded earlier and want to save cleaned versions:
# agents.to_csv(cleaned / "agents_clean.csv", index=False)
# customers.to_csv(cleaned / "customers_clean.csv", index=False)


Saved cleaned tickets to: D:\SaaS Ticket Analytics Dashboard\cleaned\tickets_master_clean.csv


In [13]:
# Cell 13 — write a simple cleaning log
log_text = f"""
Basic cleaning actions applied:
- dropped duplicates
- normalized column names (lowercase, underscores)
- parsed created_at, first_response_at, resolved_at
- derived: created_date, year_month, created_hour
- computed: resolution_time_hours, first_response_mins
- ensured sla_breached is binary (0/1)
- csat coerced to numeric and clipped 1-5 (if present)
Saved cleaned tickets to: {out_tickets}
"""
with open(cleaned / "cleaning_log.txt", "w", encoding="utf-8") as fh:
    fh.write(log_text.strip())
print("Wrote cleaning_log.txt")


Wrote cleaning_log.txt
